# TwitterStream to Azure blob / DBFS - Step 1


* [jump to Twitter-Stream-Azure notebook]($./TwitterStream to Azure blob DBFS - Step 1)
* [jump to Twitter-SentimentAnalysis notebook]($./Huggingface Sentiment Analysis Step 3)
* [Pipeline](https://adb-3234447377967728.8.azuredatabricks.net/?o=3234447377967728#joblist/pipelines/c7029259-25b6-4c56-83bd-ca0b5254db9c/updates/8035cda4-bf92-4cb5-896a-12e94ac36f3d)

In [0]:

# Mount ADLS Gen2 or Blob Storage with ABFS ## https://docs.databricks.com/dbfs/mounts.html#mount-adls-gen2-or-blob-storage-with-abfs
# first create a keyvault, then generate a secret.
# create mnt here holder dbutils.fs.mkdirs("/mnt/tweet-holder")
#https://docs.databricks.com/security/aad-storage-service-principal.html
# create a blob to hold the tweets.
## for a principal you need to set the scope. for this one it's https://adb-3234447377967728.8.azuredatabricks.net/?o=3234447377967728#secrets/createScope otherwise it's
## Go to https://<databricks-instance>#secrets/createScope 
# dbutils.secrets.listScopes() to see the name in case you forget.


In [0]:
#dbutils.secrets.listScopes()

Out[1]: [SecretScope(name='azure-storage')]

In [0]:
#dbutils.fs.refreshMounts()
#dbutils.fs.mkdirs("/mnt/tweet-holder")
#dbutils.fs.mounts()
# client id for configs is the applicationid in for my service principal xxx-ebf417f68948

Mounts successfully refreshed.
Out[37]: [MountInfo(mountPoint='/databricks-datasets', source='databricks-datasets', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-tracking', source='databricks/mlflow-tracking', encryptionType=''),
 MountInfo(mountPoint='/databricks-results', source='databricks-results', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-registry', source='databricks/mlflow-registry', encryptionType=''),
 MountInfo(mountPoint='/mnt/tweet-holder', source='abfss://twittercatcher@twitterblobber.dfs.core.windows.net/', encryptionType=''),
 MountInfo(mountPoint='/', source='DatabricksRoot', encryptionType='')]

-- for databricks file system(dbfs) https://learn.microsoft.com/en-us/azure/databricks/dbfs/mounts
  
  -- need to create service principal https://learn.microsoft.com/en-us/azure/active-directory/develop/howto-create-service-principal-portal

-- create database connector https://portal.azure.com/?quickstart=true#view/HubsExtension/BrowseResource/resourceType/Microsoft.Databricks%2FaccessConnectors


xxxx66d-6509-4d5d-xxx-cac8fa836db0
  "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="azure-storage",key="blob-storage-databricks"),

In [0]:
'''
# depreciated and didn't work
dbutils.fs.mount(
source = "wasbs://twittercatcher@twitterblobber.blob.core.windows.net",
mount_point = "/mnt/tweet-holder2",
extra_configs = {"fs.azure.account.key.twitterblobber.blob.core.windows.net":dbutils.secrets.get(scope="azure-storage",key="blob-storage-databricks")})

df = spark.read.text("/mnt/tweet-holder/test.json")

df.show()
'''

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-304508422698303>:2
      1 # depreciated and didn't work
----> 2 dbutils.fs.mount(
      3 source = "wasbs://twittercatcher@twitterblobber.blob.core.windows.net",
      4 mount_point = "/mnt/tweet-holder2",
      5 extra_configs = {"fs.azure.account.key.twitterblobber.blob.core.windows.net":dbutils.secrets.get(scope="azure-storage",key="blob-storage-databricks")})
      7 df = spark.read.text("/mnt/tweet-holder2/test.json")
      9 df.show()

File /databricks/python_shell/dbruntime/dbutils.py:362, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    360 exc.__context__ = None
    361 exc.__cause__ = None
--> 362 raise exc

ExecutionError: An error occurred while calling o435.mount.
: shaded.databricks.org.apache.hadoop.fs.azure.AzureException: hadoop_azure_shaded.com.micro

In [0]:
''' created the first time -- #used service principal, and secret, scopt didn't work for me.
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": "xxxx-b46c-402d-bed0-xx",
          "fs.azure.account.oauth2.client.secret": "xxxx",
          "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/xxx-eced-43d3-b1f2-7315f4ec788c/oauth2/token"}
dbutils.fs.mount( source = "abfss://twittercatcher@twitterblobber.dfs.core.windows.net/", mount_point = "/mnt/tweet-holder", extra_configs = configs)
'''

Out[17]: True

In [0]:
# should use databricks secrets and the CLI to store and retrieve those keys in a safe way.
#
# for a first try, you can setup you twitter keys here
consumer_key = "xxx"
consumer_secret = "xxxxZInFbkPHTxGftPi8cW0bODuogUmgWXtGpPfncT61j"
access_token = "xxx8955956233-8mU4smuEPIOCne5uqoX9jJB3R6mHOt"
access_token_secret = "bPNPvtCFuhAxNJJvlzMmaPlEDkUX3UzYaLz4HfXLxa289"

# in my demo, I read in the keys from another notebook in the cell below (which can be savely removed or commented out)


In [0]:
# %run "./Twitter-Setup"

In [0]:
#!/databricks/python3/bin/python -m pip install --upgrade pip

In [0]:
!pip install tweepy jsonpickle

     |████████████████████████████████| 102 kB 45.8 MB/s 
     |████████████████████████████████| 40 kB 6.6 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-a4e9e473-f623-43f7-8d61-f85ae879992e/bin/python -m pip install --upgrade pip' command.


In [0]:
import tweepy
import calendar
import time
import jsonpickle
import sys


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, timeout=60)
print(f'Twitter screen name: {api.verify_credentials().screen_name}')


# Subclass Stream  
class TweetStream(tweepy.Stream):

    def __init__(self, filename):
        tweepy.Stream.__init__(self, consumer_key=consumer_key, consumer_secret=consumer_secret,
                             access_token=access_token, access_token_secret=access_token_secret)
        self.filename = filename
        self.text_count = 0
        self.tweet_stack = []


    def on_status(self, status):
        #print('*'+status.text)
        self.text_count = self.text_count + 1
        self.tweet_stack.append(status)
    
        # when to print
        if (self.text_count % 1 == 0):
            print(f'retrieving tweet {self.text_count}: {status.text}')

        # how many tweets to batch into one file
        if (self.text_count % 10 == 0):
            self.write_file()
            self.tweet_stack = []

        # hard exit after collecting n tweets
        if (self.text_count == 30):
            raise Exception("Finished job")

    def write_file(self):
        file_timestamp = calendar.timegm(time.gmtime())
        fname = self.filename + '/tweets_' + str(file_timestamp) + '.json'


        f = open(fname, 'w')
        for tweet in self.tweet_stack:
            f.write(jsonpickle.encode(tweet._json, unpicklable=False) + '\n')
        f.close()
        print("Wrote local file ", fname)

    def on_error(self, status_code):
        print("Error with code ", status_code)
        sys.exit()


# Initialize instance of the subclass
tweet_stream = TweetStream("/dbfs/mnt/tweet-holder")

# Filter realtime Tweets by keyword
try:
    tweet_stream.filter(languages=["en"],
                        track=["Cub Foods", "Cub food"
                               "Minnesota Grocer"])



except Exception as e:
    print("some error ", e)
    print("Writing out tweets file before I have to exit")
    tweet_stream.write_file()
finally:
    print("Downloaded tweets ", tweet_stream.text_count)
    tweet_stream.disconnect()


Twitter screen name: devworkStill
retrieving tweet 1: RT @AndreaR9Md: It's legal to shoot their mama, you wanna tell the or should I?
retrieving tweet 2: @willystizzle4 @colinsteinke @bshallenberger @Seth_3773 Really interesting how the Cubs are moving last years short… https://t.co/xHG5Ll0S6X
retrieving tweet 3: Probably my favorite pic of @CoachPainter     Until the day he holds the natty trophy #boilerup #gocubs
retrieving tweet 4: RT @iftiinkaguriga: Some people really don’t fear Yawm al Qiyamah and it shows
retrieving tweet 5: RT @SarahHuckabee: Amazing day with my family and @ARGameandFish helping protect our bear population. Holding baby bear 🐻 cubs is unforgett…
retrieving tweet 6: @4Sullivan @bshallenberger @Seth_3773 Really interesting how the Cubs are moving last years shortstop Nico Hoerner… https://t.co/KNTTF8Giur
retrieving tweet 7: @AndreaR9Md The cubs look about as happy as her kids do.  Is this supposed to show her softer side?
retrieving tweet 8: RT @CubsViews: Seiya 

ERROR:tweepy.streaming:Stream encountered an exception
Traceback (most recent call last):
  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-a4e9e473-f623-43f7-8d61-f85ae879992e/lib/python3.9/site-packages/tweepy/streaming.py", line 94, in _connect
    self.on_data(line)
  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-a4e9e473-f623-43f7-8d61-f85ae879992e/lib/python3.9/site-packages/tweepy/streaming.py", line 418, in on_data
    return self.on_status(status)
  File "<command-2294717165822538>", line 42, in on_status
    raise Exception("Finished job")
Exception: Finished job


Downloaded tweets  30


In [0]:
dbutils.notebook.exit("stop")

# Setup Utilities

### create a DBFS directory, check for number of files, deletes a certain number of files ...

In [0]:
# create a directory to buffer the streamed data
!mkdir "/dbfs/mnt/tweet-holder/tweets"

mkdir: cannot create directory ‘/dbfs/data/tweets’: No such file or directory


In [0]:
# count files in that directory - compare with #files in DLT bronze
#!ls -l /dbfs/mnt/tweet-holder/twitter_dataeng2 | wc

In [0]:
# disk usage
#!du -h  dbfs/mnt/tweet-holder/twitter_dataeng2

In [0]:
'''# remove n files. use this to trim demo
files = dbutils.fs.ls("/data/twitter_dataeng2")
del = 400
print(f'number of files: {len(files)}')
print(f'number of files to delete: {del}')


for x, file in enumerate(files):
  # delete n files from directory
  if x < del :
    # print(x, file)
    dbutils.fs.rm(file.path)

    
# use dbutils to copy over files... 
# dbutils.fs.cp("/data/twitter_dataeng/" +f, "/data/twitter_dataeng2/")
'''